In [1]:
import pandas as pd
import json
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters
import numpy as np
import time
import re
from functools import reduce
import operator
from nltk.corpus import wordnet

In [4]:
# 필요한 데이터 준비
path = r'C:\Users\student\Documents\jsonData\VOC/'
reveiwFile = open(path+'cameraReview.json', encoding="UTF-8")
crawlData = json.loads(reveiwFile.read())
crawl = pd.DataFrame(crawlData)[['asin', 'reviewText']]

existFile = open(path+'test.json', encoding="UTF-8")
existData = json.loads(existFile.read())
exist = pd.DataFrame(existData) #[['asin', 'reviewText']]

reviewTotal = pd.concat([crawl, exist]).drop_duplicates(['reviewText']).dropna()
reviewTotal['reviewID'] = range(len(reviewTotal))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # This is added back by InteractiveShellApp.init_path()


In [5]:
exist

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime,category,index
0,B00000JDES,"[3, 3]",1.0,This camera was excellent for the price - on s...,"06 14, 2003",A1U3OF3P0VF7WB,anna meiners,Great Camera Designed to Fail?,1055548800,Point & Shoot Digital Cameras,13809
1,B00000JDES,"[1, 1]",4.0,The Agfa 780c is great for posting pictures on...,"02 13, 2002",A3CC3I9M28ONDO,Daxx,Excellent starter camera.,1013558400,Point & Shoot Digital Cameras,13809
10,B000021YUL,"[0, 0]",4.0,Bought this camera thinking that I could find ...,"11 15, 2010",A2N61S38JC08C9,Annette D. Cooper,Olympus D450,1289779200,Point & Shoot Digital Cameras,13808
100,B00004TBDI,"[0, 0]",3.0,Nice camera at first. If you take 3 pictures ...,"04 21, 2001",A1PAZNH4HEGSAG,Daniel Cardenas,It broke,987811200,Point & Shoot Digital Cameras,13807
1000,B00008IOWR,"[4, 4]",5.0,I bought this camera with wishes of quite good...,"08 23, 2003",A1PWGSNNXLWPXB,Tommy J.Iwanski,I am totally surprised BEST QUALITY! and so le...,1061596800,Point & Shoot Digital Cameras,10774
10000,B000NK6DZI,"[5, 5]",5.0,I've had this camera for a couple of weeks now...,"02 23, 2008",A2D2YF5XS1G8V3,"Setter Dog ""Jack""",Very happy with this camera,1203724800,Point & Shoot Digital Cameras,9736
10001,B000NK6DZI,"[0, 1]",1.0,I have bought this product few years back (not...,"11 8, 2012",AOJK57GR74EFD,SGIN,Expensive and Use-able,1352332800,Point & Shoot Digital Cameras,9736
10002,B000NK6DZI,"[7, 7]",5.0,The video quality of this little toy is beyond...,"12 18, 2007",A2BIGQ9AIZR95Z,Shen Chiu,Excellent camera and camcorder!!!,1197936000,Point & Shoot Digital Cameras,9736
10003,B000NK6DZI,"[1, 1]",4.0,This camera needs light. The flash is weak. ...,"11 1, 2007",AVG5J8GVBVS0P,S. Protopapas,"Aziz, light!",1193875200,Point & Shoot Digital Cameras,9736
10004,B000NK6DZI,"[5, 5]",5.0,The form factor fits nicely in the hand when t...,"04 2, 2008",A2I0XILIB0KHXY,Sunny,Great ski/snowboard camera,1207094400,Point & Shoot Digital Cameras,9736


In [12]:
def makeSents(review):
    start = time.time() 
    # sentence 구분 시 pass sign 지정
    punkt_param = PunktParameters()
    punkt_param.abbrev_types = set(['p.s','vs','etc', 'but'])
    sent_splitter = PunktSentenceTokenizer(punkt_param)
    return sent_splitter.tokenize(review)


In [46]:
def mapping(func):
    start = time.time() 
    resultSer = ser.map(lambda x: func)
    print("Runtime: %0.2f sec"%(time.time() - start))
    return resultSer

In [50]:
ser =  reviewTotal['reviewText']

In [51]:
a = mapping(makeSents(ser))

Runtime: 21.55 sec
Runtime: 0.03 sec


In [52]:
a[0]

0                                 [Final Update, 8/27/14]
1       [I've had this camera for four years now and i...
2       [I know that this is an entry level dslr, I kn...
3       [I am new to DSLR cameras., I've always just h...
4       [If you're coming from a bridge or compact cam...
5       [I used this for about 5 years and had the mai...
6       [I'm no expert or anything., Just a standard g...
7       [I think the camera is an amazing buy for the ...
8       [When I first had my son I just couldn't justi...
9       [This is my first DSLR., My photography needs ...
10      [I have nothing but good things to say about t...
11      [I am a pro and yes this is good enough for an...
12      [Bought this camera about a year ago as a gate...
14      [I love this camera., I got it on January 09, ...
15      [Nice while it worked, but now its worthless w...
18       [Professional photographer MASSIVELY IMPRESSED!]
19      [The Sony a6000 is by far the best camera that...
20      [I've 

In [ ]:
reduce(operator.concat, serSents)